<a href="https://colab.research.google.com/github/wajnryt/JerusMLDeepLearning2019/blob/master/JerusMLSimpleShapesNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/JerusML", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/JerusML


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import numpy as np
import PIL

train_set = ImageFolder(root='/content/JerusML/My Drive/JerusML/Grayscale/', transform=ToTensor())
test_set = ImageFolder(root='/content/JerusML/My Drive/JerusML/GrayscaleTEST/', transform=ToTensor())
print(data.classes)

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)

train_loader.dataset[0][0].shape

['circles', 'rectangles', 'triangles']


AttributeError: ignored

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
train_set = ImageFolder(root='/content/JerusML/My Drive/JerusML/Grayscale/', transform=ToTensor())
test_set = ImageFolder(root='/content/JerusML/My Drive/JerusML/GrayscaleTEST/', transform=ToTensor())


batch_size = 10
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)


print ('==>>> total trainning batch number: {}'.format(len(train_loader)))
print ('==>>> total testing batch number: {}'.format(len(test_loader)))

## network
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def name(self):
        return "MLP"

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1, 2)
        self.conv2 = nn.Conv2d(20, 50, 5, 1, 2)
        self.fc1 = nn.Linear(164*218*50, 3)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        print(x.shape)
        x = x.view(-1, 164*218*50)
        x = self.fc1(x)
        return x
    
    def name(self):
        return "LeNet"

## training
model = LeNet()
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    # trainning
    ave_loss = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        ave_loss = ave_loss * 0.9 + loss.data * 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print ('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(
                epoch, batch_idx+1, ave_loss))
    # testing
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        x, target = Variable(x, volatile=True), Variable(target, volatile=True)
        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        # smooth average
        ave_loss = ave_loss * 0.9 + loss.data * 0.1
        
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_loader):
            print ('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, ave_loss,float(correct_cnt)/total_cnt))

torch.save(model.state_dict(), model.name())

==>>> total trainning batch number: 210
==>>> total testing batch number: 31
torch.Size([10, 50, 164, 218])
torch.Size([10, 50, 164, 218])
torch.Size([10, 50, 164, 218])
torch.Size([10, 50, 164, 218])
torch.Size([10, 50, 164, 218])
torch.Size([10, 50, 164, 218])
torch.Size([10, 50, 164, 218])
torch.Size([10, 50, 164, 218])


In [0]:
from matplotlib import pyplot as plt
import os
import numpy as np
import glob
import cv2

import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

#!ls "/content/JerusML/My Drive/JerusML/circles"
#load circles
input_path = "/content/JerusML/My Drive/JerusML/circles/"
circles = []
circlesLabels = []
count = 1
for filename in os.listdir(input_path):
  image = cv2.imread(input_path + filename)
  #image = np.float32(image)
  circles.append(image)
  count = count + 1
print(len(circles))

if True:
  #load rectangles
  input_path = "/content/JerusML/My Drive/JerusML/rectangles/"
  count = 1
  rectangles = []
  rectanglesLabels = []
  for filename in os.listdir(input_path):
    image = cv2.imread(input_path + filename)
    #image = np.float32(image)
    rectangles.append(image)
    count = count + 1
  print(len(rectangles))

  #load triangles
  input_path = "/content/JerusML/My Drive/JerusML/triangles/"
  count = 1
  triangles = []
  trianglesLabels = []
  for filename in os.listdir(input_path):
    image = cv2.imread(input_path + filename)
    #image = np.float32(image)
    triangles.append(image)
    count = count + 1
  print(len(triangles))

print('ok')
  
  

1001
1001
1000
ok
